In [1]:
# Python-native
import os
import warnings
from pathlib import Path
from datetime import datetime

# Data manipulation
import numpy as np
import pandas as pd

# Statistics
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import brunnermunzel
import statsmodels.stats as sms
from statsmodels.stats import proportion
from statsmodels.stats import power as sms_power

In [2]:
# Path of A/B test result files
path = Path(r'D:\High-usage\Data Science\Data Analyst Path\Projects\Analytical Projects\Jooble\AB Test Results\Data Prepared to Stat Testing')

files = list(path.glob('*.csv'))
if not files:
        raise FileNotFoundError("No CSV files found in the directory.")

# Sorting files by date suffix (e.g. "jooble_ml_ab_test_data_20250804") and creation date
sorted_dated_files = sorted(
        files, 
        key=lambda f: (datetime.strptime(f.stem.split('_')[-1], '%Y%m%d'), datetime.fromtimestamp(f.stat().st_birthtime)), 
        reverse=True
)

# Fetching the most recent file path
latest_file = sorted_dated_files[0]

# Reading the latest A/B test result file
ab_test = pd.read_csv(latest_file)

In [3]:
# Statistical test functions (selected based on metric data type)

def z_test_prop(variant_metric: pd.DataFrame) -> dict:
    """Z-test for proportions with a Confidence Interval, Effect Size (Cohen's h) and Power Analysis."""
        
    metric_col_name = variant_metric.iloc[:, 1].name

    # Calculating the number of successes and observations for each variant
    sizes = variant_metric.groupby('variant').agg(
        count=(metric_col_name, 'sum'),
        nobs=(metric_col_name, 'size')
    )

    # Reorder so 'control' is first, regardless of treatment group name (e.g. 'treatment' vs. 'experiment')
    indexes = sizes.index.tolist()
    indexes.remove('control')
    new_order = ['control'] + indexes
    sizes = sizes.reindex(new_order)

    count = sizes['count'].to_list()
    nobs = sizes['nobs'].to_list()

    # Performing the z-test for proportions
    stat, pval = proportion.proportions_ztest(count, nobs, alternative='two-sided', prop_var=False)

    # Calculating the confidence interval for the difference in proportions
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            ci_low, ci_upp = proportion.confint_proportions_2indep(
                count[1], nobs[1], count[0], nobs[0], method='score' # score's root-finding algorithm may hit NaN if counts/proportions are at edge values (e.g., 0 or 1)
                                                                     # the "wald" (inaccurate) or "agresti-caffo" (accurate) methods are more robust in edge cases
            )
    except ValueError:
        # Fallback to 'agresti-caffo' if 'score' fails
        ci_low, ci_upp = proportion.confint_proportions_2indep(
            count[1], nobs[1], count[0], nobs[0], method='agresti-caffo'
        )
    
    # Power analysis for the z-test
    prop1 = count[0] / nobs[0] # control proportion
    prop2 = count[1] / nobs[1] # treatment proportion
    nobs1 = nobs[0]
    nobs2 = nobs[1]
    ratio = nobs2 / nobs1 # treatment to control ratio
    alpha = 0.05
    
    # Calculate effect size (Cohen’s h)
    # Cohen’s h measures the absolute difference between two proportions on an arcsine scale, not the relative difference; it's independent of sample size
    # The larger the absolute difference between the proportions, the larger the effect size
        # h ≈ 0.20 → Small effect (in practical terms)
        # h ≈ 0.50 → Medium effect (in practical terms)
        # h ≈ 0.80 → Large effect (in practical terms)
    effect_size = proportion.proportion_effectsize(prop2, prop1)

    # Calculate power of the test for detecting effect size with sample and alpha
    power = sms_power.zt_ind_solve_power(effect_size=effect_size, nobs1=nobs1, alpha=alpha, ratio=ratio, alternative='two-sided')

    return {
        metric_col_name: [
            # {'size': f'[c_{nobs[0]}, t_{nobs[1]}]'},
            {'prop': f'[c_{prop1:.3f}, t_{prop2:.3f}]'},
            {'rel_impr': f'{(prop2 - prop1) / prop1:.2%}'},
            {'abs_impr': f'{prop2 - prop1:.2%}'},
            {'pval': f'{pval:.4f}'}, 
            {'effect': f'{effect_size:.3f} ({'small' if abs(effect_size) < 0.20 else 'medium' if abs(effect_size) < 0.50 else 'large'})'},
            {'ci_diff': f'[{ci_low:.3f}, {ci_upp:.3f}]'}, 
            {'power': f'{power:.3f}'},
        ]
    }

def chi_squared_test(variant_metric: pd.DataFrame) -> dict:
    """Chi-squared test for categorical variables with an Effect Size (Cramer's V, Cohen's w) and Power Analysis."""
    
    catvar_col_name = variant_metric.iloc[:, 1].name

    # Create a contingency table
    contingency = pd.crosstab(variant_metric['variant'], variant_metric[catvar_col_name])
    k = contingency.shape[1] # number of categories
    n = contingency.sum().sum() # total number of observations

    # Perform the chi-squared test
    chi2, pval, dof, expected = chi2_contingency(contingency)
    
    # Calculate effect size (Cramer's V)
    # Cramer's V is a conservative measure of association between two categorical variables
        # V ≈ 0.00-0.10 → Very small effect
        # V ≈ 0.10-0.30 → Small effect
        # V ≈ 0.30-0.50 → Medium effect
        # V > 0.50 → Large effect
    # *interpretation depends on the number of categories (k) – the more categories, the smaller the effect size for the same association strength
    cramer_v = np.sqrt(chi2 / (n * min(contingency.shape) - 1))
    
    # Power analysis for the chi-squared test
    cohen_w = cramer_v * np.sqrt(k - 1) # Cramer's V to Cohen's w conversion (used when calculating power)
    alpha = 0.05
    power = sms_power.GofChisquarePower().power(effect_size=cohen_w, nobs=n, alpha=alpha, n_bins=k)
    
    return {
        catvar_col_name: [
            {'categs': contingency.columns.tolist()},
            # {'size': [contingency.sum().loc[dt] for dt in contingency.columns]},
            {'prop': [f'{variant_metric[catvar_col_name].value_counts(normalize=True).loc[dt]:.1%}' for dt in contingency.columns]},
            {'pval': f'{pval:.4f}'},
            {'effect': f'{cramer_v:.3f} ({'negligible' if cramer_v < 0.10 else 'small' if cramer_v < 0.30 else 'medium' if cramer_v < 0.50 else 'large'})'},
            {'power': f'{power:.3f}'}
        ]
    }

def brunner_munzel_test(variant_metric: pd.DataFrame) -> dict:
    """Brunner-Munzel test for normal/non-normal distributions and equal/unequal variances with an Effect Size (Pest p*/Cliff's δ)."""
    
    metric_col_name = variant_metric.iloc[:, 1].name
    
    group1 = variant_metric[variant_metric['variant'] == 'control'][metric_col_name]
    group2 = variant_metric[variant_metric['variant'] == 'treatment'][metric_col_name]
    
    # Performing the Brunner-Munzel test
    stat, pval = brunnermunzel(group1, group2, alternative='two-sided')
    
    # Calculating the effect size (Pest/p*) as the probability of group2 being greater than group1 (stochastic superiority)
    # Pest (i.e. probability estimation) serves as a rank-based probabilistic effect size that complements the Brunner-Munzel test
        # Ranges from 0 to 1
        # p* = 0.5 = no effect (called "stochastic equality") = the probability that Y is greater than X equals the probability that X is greater than Y.
        # p* > 0.5 → values in Y tend to be larger than X
        # p* < 0.5 → values in X tend to be larger than Y
    nx, ny = len(group1), len(group2)
    all_data = np.concatenate([group1, group2])
    ranks = stats.rankdata(all_data)  # Automatically handles ties
    rank_y = ranks[nx:]  # Ranks for group y
    mean_rank_y = np.mean(rank_y)
    effect_pest = (mean_rank_y - (ny + 1)/2) / nx # Derived from the original effect size (Pest) formula P(X<Y)+0.5⋅P(X=Y)

    # Convert probability-based effect size (Pest) to straightforward Cliff’s Delta (-1 to 1 where 0 indicates no effect)
        # Ranges from -1 to 1
        # δ > 0 → Y tends to dominate X (e.g. +0.5 δ = 0.75 Pest)
        # δ < 0 → X tends to dominate Y (e.g. -0.5 δ = 0.25 Pest)
        # δ ≈ 0 → No dominance (0 = 0.5 Pest = stochastic equality)
    effect_delta = 2 * effect_pest - 1 

    return {
        metric_col_name: [
            # {'size': f'[c_{len(group1)}, t_{len(group2)}]'},
            {'pval': f'{pval:.4f}'},
            {'effect': f'{np.clip(effect_delta, -1.0, 1.0):.2f} ({'c over t' if abs(effect_delta) < 0 else 'no effect' if abs(effect_delta) == 0 else 't over c'})'},
        ]
    }

In [4]:
# Extracting primary and secondary metrics
target_metrics_list = ab_test.columns[ab_test.columns.str.match(r'^(sm|pm)')].to_list()

dtype_dstat_dict = {}
# Calculating the type of statistical test for each metric based on its data type
for metric in target_metrics_list:
    variant_metric = ab_test[['variant', metric]]
    # Checking if the metric is binary
    if set(variant_metric[metric].dropna().unique()) <= set([0, 1]):
        if 'binary' in dtype_dstat_dict:
            dtype_dstat_dict['binary'].append(z_test_prop(variant_metric))
        else:
            dtype_dstat_dict['binary'] = [z_test_prop(variant_metric)]
    # Checking if the metric is object/category
    elif variant_metric[metric].dtype.kind in {'O', 'c'}:
        if 'categorical' in dtype_dstat_dict:
            dtype_dstat_dict['categorical'].append(chi_squared_test(variant_metric))
        else:
            dtype_dstat_dict['categorical'] = [chi_squared_test(variant_metric)]
    # Checking if the metric is integer/float
    elif variant_metric[metric].dtype.kind in {'i', 'f'}:
        if 'numeric' in dtype_dstat_dict:
            dtype_dstat_dict['numeric'].append(brunner_munzel_test(variant_metric))
        else:
            dtype_dstat_dict['numeric'] = [brunner_munzel_test(variant_metric)]
    else:
        raise ValueError(f"Unsupported data type for metric '{metric}': type({variant_metric[metric].dtype}), values({variant_metric[metric].unique()[:5]}).")

In [5]:
# Disassembling the dictionary into an Excel report file with proper formatting

# Necessary libraries for Excel file creation
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
import json

def disassemble_dictionary_to_excel(data_dict, output_filename='ab_test_results.xlsx'):
    """
    Disassembles a nested dictionary containing A/B test results and exports to Excel
    with proper formatting and separate sections for each data type.
    
    Args:
        data_dict (dict): The input dictionary with binary, numeric, and categorical data
        output_filename (str): Name of the output Excel file
    """
    
    # Create a new workbook
    wb = Workbook()
    ws = wb.active
    ws.title = "AB Test Results"
    
    # Define styles
    header_font = Font(bold=True, size=12, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    
    metric_font = Font(bold=True, size=11, color='FFFFFF')
    metric_fill = PatternFill(start_color='4F81BD', end_color='4F81BD', fill_type='solid')
    
    stat_font = Font(size=10)
    stat_fill = PatternFill(start_color='DCE6F1', end_color='DCE6F1', fill_type='solid')
    
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    center_alignment = Alignment(horizontal='center', vertical='center')
    
    current_row = 1
    
    # Process each data type (binary, numeric, categorical)
    for data_type, metrics_list in data_dict.items():
        # Add section header
        section_header = f"{data_type.upper()} VARIABLES"
        ws.cell(row=current_row, column=1, value=section_header)
        ws.cell(row=current_row, column=1).font = header_font
        ws.cell(row=current_row, column=1).fill = header_fill
        ws.cell(row=current_row, column=1).border = border
        ws.cell(row=current_row, column=1).alignment = center_alignment
        
        # Merge cells for section header (assuming 3 columns width)
        ws.merge_cells(f'A{current_row}:C{current_row}')
        current_row += 2
        
        # Process each metric in this data type
        for metric_dict in metrics_list:
            for metric_name, stats_list in metric_dict.items():
                # Add metric name as table header
                ws.cell(row=current_row, column=1, value="Metric")
                ws.cell(row=current_row, column=2, value=metric_name)
                
                # Style metric header
                ws.cell(row=current_row, column=1).font = metric_font
                ws.cell(row=current_row, column=1).fill = metric_fill
                ws.cell(row=current_row, column=1).border = border
                ws.cell(row=current_row, column=1).alignment = center_alignment
                
                ws.cell(row=current_row, column=2).font = metric_font
                ws.cell(row=current_row, column=2).fill = metric_fill
                ws.cell(row=current_row, column=2).border = border
                ws.cell(row=current_row, column=2).alignment = center_alignment
                
                current_row += 1
                
                # Add statistics for this metric
                for stat_dict in stats_list:
                    for stat_name, stat_value in stat_dict.items():
                        ws.cell(row=current_row, column=1, value=stat_name)
                        ws.cell(row=current_row, column=2, value=stat_value)
                        
                        # Style statistics rows
                        ws.cell(row=current_row, column=1).font = stat_font
                        ws.cell(row=current_row, column=1).fill = stat_fill
                        ws.cell(row=current_row, column=1).border = border
                        ws.cell(row=current_row, column=1).alignment = center_alignment
                        
                        ws.cell(row=current_row, column=2).font = stat_font
                        ws.cell(row=current_row, column=2).border = border
                        
                        current_row += 1
                
                # Add empty row between metrics for better readability
                current_row += 1
        
        # Add extra space between sections
        current_row += 2
    
    # Adjust column widths
    ws.column_dimensions['A'].width = 25
    ws.column_dimensions['B'].width = 30
    ws.column_dimensions['C'].width = 15
    
    # Save the workbook
    full_path = os.path.abspath(output_filename)
    wb.save(output_filename)
    print(f"Excel file '{output_filename}' has been created successfully!")
    print(f"Full path: {full_path}")
    
    return output_filename

def create_summary_sheet(data_dict, output_filename='ab_test_results_report.xlsx'):
    """
    Creates an enhanced version with both detailed and summary sheets
    """
    
    # Create a new workbook with summary
    wb = Workbook()
    
    # Remove default sheet and create our sheets
    wb.remove(wb.active)
    summary_ws = wb.create_sheet("Summary")
    detailed_ws = wb.create_sheet("Detailed Results")
    
    # Set Summary sheet tab color to green-grey
    summary_ws.sheet_properties.tabColor = "A8C090"
    
    # Create summary table
    summary_data = []
    
    for data_type, metrics_list in data_dict.items():
        for metric_dict in metrics_list:
            for metric_name, stats_list in metric_dict.items():
                row_data = {'Data Type': data_type.capitalize(), 'Metric': metric_name}
                
                # Extract key statistics
                for stat_dict in stats_list:
                    for stat_name, stat_value in stat_dict.items():
                        row_data[stat_name] = stat_value
                
                summary_data.append(row_data)
    
    # Create DataFrame and write to summary sheet
    if summary_data:
        df_summary = pd.DataFrame(summary_data)
        
        # Write headers
        headers = list(df_summary.columns)
        for col_idx, header in enumerate(headers, 1):
            cell = summary_ws.cell(row=1, column=col_idx, value=header)
            cell.font = Font(bold=True, color='FFFFFF')
            cell.fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
            cell.border = Border(
                left=Side(style='thin'),
                right=Side(style='thin'),
                top=Side(style='thin'),
                bottom=Side(style='thin')
            )
            cell.alignment = Alignment(horizontal='center', vertical='center')
        
        # Write data
        for row_idx, row in enumerate(df_summary.itertuples(index=False), 2):
            for col_idx, value in enumerate(row, 1):
                cell = summary_ws.cell(row=row_idx, column=col_idx, value=value)
                cell.border = Border(
                    left=Side(style='thin'),
                    right=Side(style='thin'),
                    top=Side(style='thin'),
                    bottom=Side(style='thin')
                )
                if row_idx % 2 == 0:
                    cell.fill = PatternFill(start_color='F2F2F2', end_color='F2F2F2', fill_type='solid')
        
        # Auto-adjust column widths
        for column in summary_ws.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 30)
            summary_ws.column_dimensions[column_letter].width = adjusted_width
    
    # Now create the detailed sheet with the same formatting as before
    current_row = 1
    
    # Define styles (same as before)
    header_font = Font(bold=True, size=12, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    
    metric_font = Font(bold=True, size=11, color='FFFFFF')
    metric_fill = PatternFill(start_color='4F81BD', end_color='4F81BD', fill_type='solid')
    
    stat_font = Font(size=10)
    stat_fill = PatternFill(start_color='DCE6F1', end_color='DCE6F1', fill_type='solid')
    
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    
    center_alignment = Alignment(horizontal='center', vertical='center')
    
    # Define test names for each data type
    test_names = {
        'binary': 'Z-test for proportions',
        'categorical': 'Chi-squared test',
        'numeric': 'Brunner-Munzel test'
    }
    
    # Process each data type for detailed sheet
    for data_type, metrics_list in data_dict.items():
        # Add section header with test name
        test_name = test_names.get(data_type, '')
        section_header = f"{data_type.upper()} VARIABLES ({test_name})"
        detailed_ws.cell(row=current_row, column=1, value=section_header)
        detailed_ws.cell(row=current_row, column=1).font = header_font
        detailed_ws.cell(row=current_row, column=1).fill = header_fill
        detailed_ws.cell(row=current_row, column=1).border = border
        detailed_ws.cell(row=current_row, column=1).alignment = center_alignment
        
        # Merge cells for section header
        detailed_ws.merge_cells(f'A{current_row}:C{current_row}')
        current_row += 2
        
        # Process each metric
        for metric_dict in metrics_list:
            for metric_name, stats_list in metric_dict.items():
                # Add metric name as table header
                detailed_ws.cell(row=current_row, column=1, value="Metric")
                detailed_ws.cell(row=current_row, column=2, value=metric_name)
                
                # Style metric header
                detailed_ws.cell(row=current_row, column=1).font = metric_font
                detailed_ws.cell(row=current_row, column=1).fill = metric_fill
                detailed_ws.cell(row=current_row, column=1).border = border
                detailed_ws.cell(row=current_row, column=1).alignment = center_alignment
                
                detailed_ws.cell(row=current_row, column=2).font = metric_font
                detailed_ws.cell(row=current_row, column=2).fill = metric_fill
                detailed_ws.cell(row=current_row, column=2).border = border
                detailed_ws.cell(row=current_row, column=2).alignment = center_alignment
                
                current_row += 1
                
                # Add statistics
                for stat_dict in stats_list:
                    for stat_name, stat_value in stat_dict.items():
                        detailed_ws.cell(row=current_row, column=1, value=stat_name)
                        detailed_ws.cell(row=current_row, column=2, value=stat_value)
                        
                        # Style statistics rows
                        detailed_ws.cell(row=current_row, column=1).font = stat_font
                        detailed_ws.cell(row=current_row, column=1).fill = stat_fill
                        detailed_ws.cell(row=current_row, column=1).border = border
                        detailed_ws.cell(row=current_row, column=1).alignment = center_alignment
                        
                        detailed_ws.cell(row=current_row, column=2).font = stat_font
                        detailed_ws.cell(row=current_row, column=2).border = border
                        
                        current_row += 1
                
                current_row += 1
        
        current_row += 2
    
    # Adjust column widths for detailed sheet
    detailed_ws.column_dimensions['A'].width = 25
    detailed_ws.column_dimensions['B'].width = 30
    detailed_ws.column_dimensions['C'].width = 15
    
    # Save the workbook
    full_path = os.path.abspath(output_filename)
    wb.save(output_filename)
    print(f"Enhanced Excel file '{output_filename}' with summary and detailed sheets has been created!")
    print(f"Full path: {full_path}")
    
    return output_filename

if __name__ == "__main__":
    
    data = dtype_dstat_dict
    
    # Create the detailed results Excel file
    create_summary_sheet(data)
    
    print("\nExcel file created:")
    print("ab_test_results_report.xlsx - Multiple sheets with summary and detailed views")

Enhanced Excel file 'ab_test_results_report.xlsx' with summary and detailed sheets has been created!
Full path: d:\High-usage\Data Science\Data Analyst Path\Projects\Analytical Projects\Jooble\AB Test Results\ab_test_results_report.xlsx

Excel file created:
ab_test_results_report.xlsx - Multiple sheets with summary and detailed views
